# Imports/Downloads

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

# Loading Data

In [2]:
train = pd.read_csv('/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv')
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
sample = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')

# Cleaning Data

In [3]:
#Cleaning
train = train[['text', 'label']].copy()
sample['id'] = test['id']
test = test.drop(columns=['id', 'prompt_id'])

dev_x = train.drop(columns='label')
dev_y = train['label']

In [4]:
#Vectorizing
vector = TfidfVectorizer(stop_words='english')
vector.fit(dev_x['text'])
dev_x = pd.DataFrame(vector.transform(dev_x['text']).toarray())
test = pd.DataFrame(vector.transform(test['text']).toarray())

# Modeling

In [5]:
#XGB
classifier = XGBClassifier(objective= 'binary:logistic', n_estimators=100, n_jobs=-1)

In [6]:
%%time

classifier.fit(dev_x, dev_y)

CPU times: user 1h 47min 49s, sys: 25.2 s, total: 1h 48min 14s
Wall time: 30min 29s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=-1,
              num_parallel_tree=None, random_state=None, ...)

# Predictions

In [7]:
sample['generated'] = classifier.predict_proba(test)[:, 1]

In [8]:
sample.to_csv('submission.csv', index=False)